In [ ]:
import random
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import pytz

In [ ]:
def generate_time():
    hours = random.randint(0, 23)
    minutes = random.randint(0, 59)
    return hours, minutes

def add_times(h1, m1, h2, m2):
    h_total = h1 + h2
    m_total = m1 + m2
    
    if m_total >= 60:
        h_total += 1
        m_total -= 60
        
    h_total %= 24
    return h_total, m_total

def subtract_times(h1, m1, h2, m2):
    m_diff = m1 - m2
    h_diff = h1 - h2

    # If the minutes end up being negative, adjust hours
    if m_diff < 0:
        h_diff -= 1
        m_diff += 60

    # Adjust for 24-hour format for hours
    if h_diff < 0:
        h_diff += 24
        
    return h_diff, m_diff

def generate_choices(correct_hour, correct_minute, operation):
    choices = [(correct_hour, correct_minute)]
    
    while len(choices) < 4:
        h_shift = random.randint(1, 3)  # adjust these values for wider range
        m_shift = random.randint(1, 20)  # adjust these values for wider range
        
        if random.choice([True, False]):
            if operation == "+":
                h, m = add_times(correct_hour, correct_minute, h_shift, m_shift)
            else:
                h, m = subtract_times(correct_hour, correct_minute, h_shift, m_shift)
        else:
            if operation == "+":
                h, m = subtract_times(correct_hour, correct_minute, h_shift, m_shift)
            else:
                h, m = add_times(correct_hour, correct_minute, h_shift, m_shift)

        if (h, m) not in choices:
            choices.append((h, m))
            
    random.shuffle(choices)
    return choices

df_24h = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500):
    h1, m1 = generate_time()
    h2, m2 = generate_time()
    
    operation = random.choice(['+', '-'])
    if operation == '+':
        result_hour, result_minute = add_times(h1, m1, h2, m2)
    else:
        result_hour, result_minute = subtract_times(h1, m1, h2, m2)
    
    question = f"What is {h1:02}:{m1:02} {operation} {h2:02}:{m2:02}?"

    choices = generate_choices(result_hour, result_minute, operation)
    correct_index = choices.index((result_hour, result_minute))
    
    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_24h = df_24h.append({
        'Question': question,
        'Option A': f"{choices[0][0]:02}:{choices[0][1]:02}",
        'Option B': f"{choices[1][0]:02}:{choices[1][1]:02}",
        'Option C': f"{choices[2][0]:02}:{choices[2][1]:02}",
        'Option D': f"{choices[3][0]:02}:{choices[3][1]:02}",
        'Answer': correct_answer
    }, ignore_index=True)
df_24h = df_24h.drop_duplicates(subset=['Question'])

In [ ]:
df_24h.to_csv("df_24h.csv", index=False)

In [ ]:
def subtract_times(h1, m1, h2, m2):
    m_diff = m1 - m2
    h_diff = h1 - h2

    # If the minutes end up being negative, adjust hours
    if m_diff < 0:
        h_diff -= 1
        m_diff += 60

    # Adjust for 24-hour format for hours
    if h_diff < 0:
        h_diff += 24
        
    return h_diff, m_diff

h_diff, m_diff = subtract_times(14, 38, 4, 38)
print(f"Result: {h_diff}:{m_diff:02}")

In [ ]:
def generate_12_hour_time():
    hours = random.randint(1, 12)
    minutes = random.randint(0, 59)
    period = random.choice(['AM', 'PM'])
    return hours, minutes, period

def add_times(h1, m1, period1, h2, m2):
    if period1 == 'PM' and h1 != 12:
        h1 += 12
    if h1 == 12 and period1 == 'AM':
        h1 = 0

    total_minutes1 = h1 * 60 + m1
    total_minutes2 = h2 * 60 + m2

    total = total_minutes1 + total_minutes2
    h_total = total // 60
    m_total = total % 60

    # Ensure it's within 24-hour boundary
    h_total %= 24

    period = 'AM'
    if h_total >= 12:
        h_total -= 12
        period = 'PM'
    if h_total == 0:
        h_total = 12
    return h_total, m_total, period

def subtract_times(h1, m1, period1, h2, m2):
    if period1 == 'PM' and h1 != 12:
        h1 += 12
    if h1 == 12 and period1 == 'AM':
        h1 = 0

    total_minutes1 = h1 * 60 + m1
    total_minutes2 = h2 * 60 + m2

    diff = total_minutes1 - total_minutes2
    diff %= (24 * 60)  # Ensure positive result
    
    h_diff = diff // 60
    m_diff = diff % 60

    period = 'AM'
    if h_diff >= 12:
        h_diff -= 12
        period = 'PM'
    if h_diff == 0:
        h_diff = 12
    return h_diff, m_diff, period

def generate_choices(correct_hour, correct_minute, correct_period):
    choices = [(correct_hour, correct_minute, correct_period)]
    while len(choices) < 4:
        h_shift = random.randint(1, 2)
        m_shift = random.randint(1, 20)
        if random.choice([True, False]):
            h, m, p = add_times(correct_hour, correct_minute, correct_period, h_shift, m_shift)
        else:
            h, m, p = subtract_times(correct_hour, correct_minute, correct_period, h_shift, m_shift)
        if (h, m, p) not in choices:
            choices.append((h, m, p))
    random.shuffle(choices)
    return choices

df_12h = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500): 
    h1, m1, period1 = generate_12_hour_time()
    h2, m2 = random.randint(1, 12), random.randint(0, 59)
    
    if random.choice(['+', '-']) == '+':
        result_hour, result_minute, result_period = add_times(h1, m1, period1, h2, m2)
        question = f"What is {h1:02}:{m1:02} {period1} + {h2:02}:{m2:02}?"
    else:
        result_hour, result_minute, result_period = subtract_times(h1, m1, period1, h2, m2)
        question = f"What is {h1:02}:{m1:02} {period1} − {h2:02}:{m2:02}?"
    
    choices = generate_choices(result_hour, result_minute, result_period)
    correct_index = choices.index((result_hour, result_minute, result_period))
    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_12h = df_12h.append({
        'Question': question,
        'Option A': f"{choices[0][0]:02}:{choices[0][1]:02} {choices[0][2]}",
        'Option B': f"{choices[1][0]:02}:{choices[1][1]:02} {choices[1][2]}",
        'Option C': f"{choices[2][0]:02}:{choices[2][1]:02} {choices[2][2]}",
        'Option D': f"{choices[3][0]:02}:{choices[3][1]:02} {choices[3][2]}",
        'Answer': correct_answer
    }, ignore_index=True)
df_12h = df_12h.drop_duplicates(subset=['Question'])

In [ ]:
import random
import pandas as pd

def generate_year():
    return random.randint(1000, 2023)

def next_year(year, operation, years=1):
    return year + years if operation == "following" else year - years

def generate_choices(correct_year):
    choices = [correct_year]
    while len(choices) < 4:
        year_shift = random.randint(1, 50)  # can adjust this for wider/narrower range
        choice = correct_year + random.choice([-year_shift, year_shift])
        if choice not in choices:
            choices.append(choice)
    random.shuffle(choices)
    return choices

df_year = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500): 
    year = generate_year()
    operation = random.choice(["following", "before"])
    shift_years = random.randint(1, 20)  # Randomly choose years between 1 to 20
    correct_year = next_year(year, operation, shift_years)
    
    if operation == "following":
        question = f"Which year comes {shift_years} years after {year}?"
    else:
        question = f"Which year was {shift_years} years before {year}?"
    
    choices = generate_choices(correct_year)
    correct_index = choices.index(correct_year)
    
    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_year = df_year.append({
        'Question': question,
        'Option A': choices[0],
        'Option B': choices[1],
        'Option C': choices[2],
        'Option D': choices[3],
        'Answer': correct_answer
    }, ignore_index=True)
df_year = df_year.drop_duplicates(subset=['Question'])

In [ ]:
import random
import pandas as pd

MONTHS = [
    "January", "February", "March", "April", "May", "June", 
    "July", "August", "September", "October", "November", "December"
]

def generate_month():
    return random.choice(MONTHS)

def next_month(month, operation, months=1):
    index = MONTHS.index(month)
    if operation == "following":
        return MONTHS[(index + months) % 12]
    else:
        return MONTHS[(index - months) % 12]

def generate_choices(correct_month):
    choices = [correct_month]
    while len(choices) < 4:
        choice = random.choice(MONTHS)
        if choice not in choices:
            choices.append(choice)
    random.shuffle(choices)
    return choices

df_month = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(500):
    month = generate_month()
    operation = random.choice(["following", "before"])
    shift_months = random.randint(1, 6)  # Randomly choose months between 1 to 6
    correct_month = next_month(month, operation, shift_months)
    
    if operation == "following":
        question = f"Which month comes {shift_months} months after {month}?"
    else:
        question = f"Which month was {shift_months} months before {month}?"
    
    choices = generate_choices(correct_month)
    correct_index = choices.index(correct_month)
    
    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_month = df_month.append({
        'Question': question,
        'Option A': choices[0],
        'Option B': choices[1],
        'Option C': choices[2],
        'Option D': choices[3],
        'Answer': correct_answer
    }, ignore_index=True)

df_month = df_month.drop_duplicates(subset=['Question'])

In [ ]:
MONTHS = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]

def generate_date():
    year = random.randint(1000, 2023)
    month_index = random.randint(0, 11)
    return month_index, year

def add_years_months(month_index, year, delta_years, delta_months):
    new_month_index = month_index + delta_months
    new_year = year + delta_years
    while new_month_index > 11:
        new_year += 1
        new_month_index -= 12
    return new_month_index, new_year

def generate_choices(correct_month_index, correct_year):
    choices = [(correct_month_index, correct_year)]
    while len(choices) < 4:
        month_shift = random.randint(1, 3)
        year_shift = random.randint(1, 3)
        new_month_index, new_year = add_years_months(correct_month_index, correct_year, year_shift, month_shift)
        if (new_month_index, new_year) not in choices:
            choices.append((new_month_index, new_year))
    random.shuffle(choices)
    return choices

df_month_year = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500):  # Generate 10 questions
    month_index, year = generate_date()
    delta_years = random.randint(1, 50)
    delta_months = random.randint(1, 11)
    
    correct_month_index, correct_year = add_years_months(month_index, year, delta_years, delta_months)
    question = f"What will be the time {delta_years} years and {delta_months} months after {MONTHS[month_index]} {year}?"
    
    choices = generate_choices(correct_month_index, correct_year)
    correct_index = choices.index((correct_month_index, correct_year))
    
    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_month_year = df_month_year.append({
        'Question': question,
        'Option A': f"{MONTHS[choices[0][0]]} {choices[0][1]}",
        'Option B': f"{MONTHS[choices[1][0]]} {choices[1][1]}",
        'Option C': f"{MONTHS[choices[2][0]]} {choices[2][1]}",
        'Option D': f"{MONTHS[choices[3][0]]} {choices[3][1]}",
        'Answer': correct_answer
    }, ignore_index=True)
df_month_year = df_month_year.drop_duplicates(subset=['Question'])

In [ ]:
def generate_date():
    year = random.randint(1000, 2023)
    month = random.randint(1, 12)
    # to handle month ends, like February
    if month == 2:
        day = random.randint(1, 28)
    elif month in [4, 6, 9, 11]:
        day = random.randint(1, 30)
    else:
        day = random.randint(1, 31)
    return date(year, month, day)

def generate_choices(correct_date):
    choices = [correct_date]
    
    while len(choices) < 4:
        delta_days = random.randint(1, 30)
        wrong_date = correct_date + timedelta(days=delta_days)
        
        if wrong_date not in choices:
            choices.append(wrong_date)
            
    random.shuffle(choices)
    return choices

df_date_op = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500):  # Generate 10 questions
    start_date = generate_date()
    beta = random.randint(1, 365)  # adjust the maximum range if necessary
    operation = random.choice(['add', 'subtract'])
    if operation == 'add':
        new_date = start_date + timedelta(days=beta)
    else:
        new_date = start_date - timedelta(days=beta)

    question = f"If you {operation} {beta} days to the date {start_date.strftime('%m-%d-%Y')}, what will be the new date?"

    choices = generate_choices(new_date)
    correct_index = choices.index(new_date)

    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_date_op = df_date_op.append({
        'Question': question,
        'Option A': choices[0].strftime('%m-%d-%Y'),
        'Option B': choices[1].strftime('%m-%d-%Y'),
        'Option C': choices[2].strftime('%m-%d-%Y'),
        'Option D': choices[3].strftime('%m-%d-%Y'),
        'Answer': correct_answer
    }, ignore_index=True)
df_date_op = df_date_op.drop_duplicates(subset=['Question'])

In [ ]:
def generate_date():
    year = random.randint(1000, 2023)
    month = random.randint(1, 12)
    if month == 2:
        day = random.randint(1, 28)
    elif month in [4, 6, 9, 11]:
        day = random.randint(1, 30)
    else:
        day = random.randint(1, 31)
    return date(year, month, day)

def generate_choices(correct_date):
    choices = [correct_date]
    
    while len(choices) < 4:
        delta_days = random.randint(1, 31)
        wrong_date = correct_date + timedelta(days=delta_days)
        
        if wrong_date not in choices:
            choices.append(wrong_date)
            
    random.shuffle(choices)
    return choices

df_month_day = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500): 
    start_date = generate_date()
    alpha = random.randint(1, 12)
    beta = random.randint(1, 28)
    operation = random.choice(['add', 'subtract'])
    
    if operation == 'add':
        new_date = start_date + relativedelta(months=alpha) + timedelta(days=beta)
    else:
        new_date = start_date - relativedelta(months=alpha) - timedelta(days=beta)
    
    question = f"If you {operation} {alpha} months and {beta} days to the date {start_date.strftime('%m-%d-%Y')}, what will be the new date?"

    choices = generate_choices(new_date)
    correct_index = choices.index(new_date)

    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_month_day = df_month_day.append({
        'Question': question,
        'Option A': choices[0].strftime('%m-%d-%Y'),
        'Option B': choices[1].strftime('%m-%d-%Y'),
        'Option C': choices[2].strftime('%m-%d-%Y'),
        'Option D': choices[3].strftime('%m-%d-%Y'),
        'Answer': correct_answer
    }, ignore_index=True)
df_month_day = df_month_day.drop_duplicates(subset=['Question'])

In [ ]:
def generate_date():
    year = random.randint(1000, 2023)
    month = random.randint(1, 12)
    if month == 2:
        day = random.randint(1, 28)
    elif month in [4, 6, 9, 11]:
        day = random.randint(1, 30)
    else:
        day = random.randint(1, 31)
    return date(year, month, day)

def generate_choices(correct_date):
    choices = [correct_date]
    
    while len(choices) < 4:
        delta_days = random.randint(1, 7)
        wrong_date = correct_date + timedelta(days=delta_days)
        
        if wrong_date not in choices:
            choices.append(wrong_date)
            
    random.shuffle(choices)
    return choices

df_week_day = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500):
    start_date = generate_date()
    delta_weeks = random.randint(1, 52)
    beta = random.randint(1, 7) 
    operation = random.choice(['add', 'subtract'])
    
    if operation == 'add':
        new_date = start_date + timedelta(weeks=delta_weeks) + timedelta(days=beta)
    else:
        new_date = start_date - timedelta(weeks=delta_weeks) - timedelta(days=beta)
    
    question = f"If you {operation} {delta_weeks} weeks and {beta} days to the date {start_date.strftime('%m-%d-%Y')}, what will be the new date?"

    choices = generate_choices(new_date)
    correct_index = choices.index(new_date)

    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_week_day = df_week_day.append({
        'Question': question,
        'Option A': choices[0].strftime('%m-%d-%Y'),
        'Option B': choices[1].strftime('%m-%d-%Y'),
        'Option C': choices[2].strftime('%m-%d-%Y'),
        'Option D': choices[3].strftime('%m-%d-%Y'),
        'Answer': correct_answer
    }, ignore_index=True)
df_week_day = df_week_day.drop_duplicates(subset=['Question'])

In [ ]:
def generate_date():
    year = random.randint(1000, 2023)
    month = random.randint(1, 12)
    if month == 2:
        day = random.randint(1, 28)
    elif month in [4, 6, 9, 11]:
        day = random.randint(1, 30)
    else:
        day = random.randint(1, 31)
    return date(year, month, day)

def generate_choices(correct_week, year):
    choices = [correct_week]
    
    while len(choices) < 4:
        wrong_week = random.randint(1, 52)
        
        if wrong_week not in choices:
            choices.append(wrong_week)
            
    random.shuffle(choices)
    return choices

df_week_in_year = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500): 
    d = generate_date()
    correct_week = d.isocalendar()[1]
    
    question = f"In which week of year {d.year} does the date {d.strftime('%m-%d-%Y')} occur?"

    choices = generate_choices(correct_week, d.year)
    correct_index = choices.index(correct_week)

    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[correct_index]
    
    df_week_in_year = df_week_in_year.append({
        'Question': question,
        'Option A': f"Week {choices[0]}",
        'Option B': f"Week {choices[1]}",
        'Option C': f"Week {choices[2]}",
        'Option D': f"Week {choices[3]}",
        'Answer': correct_answer
    }, ignore_index=True)
df_week_in_year = df_week_in_year.drop_duplicates(subset=['Question'])

In [ ]:
def get_datetime_in_new_zone(dt, start_zone, target_zone):
    local = pytz.timezone(start_zone)
    dt_local = local.localize(dt, is_dst=None)
    dt_target = dt_local.astimezone(pytz.timezone(target_zone))
    return dt_target

df_time_zone = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

zones = [
    "US/Eastern",  # EST
    "UTC",
    "Australia/Sydney",  # AEDT
    "US/Pacific",  # PST
    "Asia/Kolkata",  # IST
    "Europe/Paris",  # CET
    "Europe/Athens",  # EET
    "Asia/Singapore",  # SGT
    "Asia/Shanghai",  # CST
    "Asia/Tokyo",  # JST
    "Europe/London",  # BST
    "Asia/Riyadh",  # AST
    "US/Mountain",  # MST
    "US/Hawaii"  # HST
]

for _ in range(500):
    year = random.randint(1000, 2023)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    hour = random.randint(0, 23)

    start_dt = datetime(year, month, day, hour, 0)
    start_zone = random.choice(zones)
    target_zone = random.choice([z for z in zones if z != start_zone])

    target_dt = get_datetime_in_new_zone(start_dt, start_zone, target_zone)

    question = f"If it’s {start_dt.strftime('%I %p on %B %d, %Y').replace(' 0', ' ')} in {start_zone}, what's the date and time in {target_zone}?"

    # Generate choices
    correct_answer = target_dt.strftime('%I %p on %B %d, %Y').replace(' 0', ' ')
    choices = [correct_answer]

    for _ in range(3):
        delta_hours = random.randint(1, 12)
        wrong_dt = target_dt + timedelta(hours=delta_hours)
        wrong_answer = wrong_dt.strftime('%I %p on %B %d, %Y').replace(' 0', ' ')
        choices.append(wrong_answer)

    random.shuffle(choices)
    correct_index = choices.index(correct_answer)
    answers = ['A', 'B', 'C', 'D']
    answer_letter = answers[correct_index]

    df_time_zone = df_time_zone.append({
        'Question': question,
        'Option A': choices[0],
        'Option B': choices[1],
        'Option C': choices[2],
        'Option D': choices[3],
        'Answer': answer_letter
    }, ignore_index=True)
df_time_zone = df_time_zone.drop_duplicates(subset=['Question'])

In [ ]:
start_dt = datetime(1763, 7, 7, 5, 0)
start_zone = "US/Eastern"
target_zone = "Asia/Singapore"
get_datetime_in_new_zone(start_dt, start_zone, target_zone)

In [ ]:
def days_to_hours(days):
    return days * 24

def seconds_to_hours(seconds):
    return seconds / 3600

def days_to_minutes(days):
    return days * 24 * 60

def minutes_to_hours(minutes):
    return minutes / 60

def add_times(m1, s1, m2, s2):
    total_seconds = s1 + s2
    total_minutes = m1 + m2 + (total_seconds // 60)
    remaining_seconds = total_seconds % 60
    return total_minutes, remaining_seconds

def subtract_times(h, m, m_sub, s_sub):
    total_seconds = h * 60 * 60 + m * 60
    subtract_seconds = m_sub * 60 + s_sub
    
    final_seconds = total_seconds - subtract_seconds
    final_minutes = final_seconds // 60
    remaining_seconds = final_seconds % 60
    return final_minutes, remaining_seconds

df_time_op = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])

for _ in range(1500): 
    question_type = random.randint(1, 6)
    if question_type == 1:
        days = random.randint(1, 7)
        result = days_to_minutes(days)
        question = f"Convert {days} days into minutes."
    elif question_type == 2:
        mins = random.randint(10, 500)
        result = minutes_to_hours(mins)
        question = f"Convert {mins} minutes into hours."
    elif question_type == 3:
        days = random.randint(1, 7)
        result = days_to_hours(days)
        question = f"Convert {days} days into hours."
    elif question_type == 4:
        secs = random.randint(60, 3600*5)  # Up to 5 hours worth of seconds
        result = seconds_to_hours(secs)
        question = f"Convert {secs} seconds into hours."
    elif question_type == 5:
        m1, s1 = random.randint(0, 59), random.randint(0, 59)
        m2, s2 = random.randint(0, 59), random.randint(0, 59)
        result_m, result_s = add_times(m1, s1, m2, s2)
        result = (result_m, result_s)
        question = f"Add {m1} minutes {s1} seconds and {m2} minutes {s2} seconds."
    else:
        h = random.randint(1, 3)
        m = random.randint(0, 59)
        m_sub, s_sub = random.randint(0, m if h == 1 else 59), random.randint(0, 59)
        result_m, result_s = subtract_times(h, m, m_sub, s_sub)
        result = (result_m, result_s)
        question = f"Subtract {m_sub} minutes {s_sub} seconds from {h} hours {m} minutes."

    # Generate choices
    choices = [result]
    while len(choices) < 4:
        shift = random.randint(-10, 10)
        if question_type in [1, 2, 3, 4]:
            choice = result + shift
        else:
            choice = (result[0] + shift, result[1] + shift)
        if choice not in choices:
            choices.append(choice)
    random.shuffle(choices)

    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[choices.index(result)]

    df_time_op = df_time_op.append({
        'Question': question,
        'Option A': choices[0] if question_type in [1, 2, 3, 4] else f"{choices[0][0]} minutes {choices[0][1]} seconds",
        'Option B': choices[1] if question_type in [1, 2, 3, 4] else f"{choices[1][0]} minutes {choices[1][1]} seconds",
        'Option C': choices[2] if question_type in [1, 2, 3, 4] else f"{choices[2][0]} minutes {choices[2][1]} seconds",
        'Option D': choices[3] if question_type in [1, 2, 3, 4] else f"{choices[3][0]} minutes {choices[3][1]} seconds",
        'Answer': correct_answer
    }, ignore_index=True)
df_time_op = df_time_op.drop_duplicates(subset=['Question'])

In [ ]:
def generate_time_arithmetic_questions():
    #question_types = ["vacation_days", "school_duration", "medicine_times", "homework_times", "flight_times", "rest_walk", "transportation"]
    #question_type = random.choice(question_types)
    question_type = "transportation"
    #entities = ["a boy", "a girl", "a man", "a woman"]
    #entity = random.choice(entities)
    
    def random_date_in_current_year():
        start_of_year = date(date.today().year, 1, 1)
        end_of_year = date(date.today().year, 12, 31)
        delta = end_of_year - start_of_year
        random_days = random.randint(0, delta.days)
        return start_of_year + timedelta(days=random_days)
    
    def age_difference(start_age_years, start_age_months, end_age_years, end_age_months):
        total_start_months = start_age_years * 12 + start_age_months
        total_end_months = end_age_years * 12 + end_age_months
        diff_months = total_end_months - total_start_months
        years = diff_months // 12
        months = diff_months % 12
        return years, months

    if question_type == "vacation_days":
        start_date = random_date_in_current_year()
        leave_days = random.randint(1, 30)
        end_date = start_date + timedelta(days=leave_days)
        question = f"If a person takes a leave of {leave_days} days starting from {start_date.strftime('%A, %d %B')}, on which day may the leave end?"
        answer = end_date.strftime('%A, %d %B %Y')
        wrong_answers = set()
        while len(wrong_answers) < 3:
            wrong_end_date = end_date + timedelta(days=random.randint(1, 30) * random.choice([-1, 1])) # Alter the range and multiplier to get a varied set of incorrect answers
            if wrong_end_date.strftime('%A, %d %B %Y') != answer:
                wrong_answers.add(wrong_end_date.strftime('%A, %d %B %Y'))
        wrong_answers = list(wrong_answers)
    
    elif question_type == "school_duration":
        start_age_years = random.randint(3, 7)
        start_age_months = random.randint(1, 11)
        end_age_years = random.randint(start_age_years + 1, 14)
        end_age_months = random.randint(0, 11)
        #pronoun = "he" if entity in ["a boy", "a man"] else "she"
        years, months = age_difference(start_age_years, start_age_months, end_age_years, end_age_months)
        question = f"If a person was {start_age_years} years {start_age_months} month(s) old when he joined school and now he is {end_age_years} years {end_age_months} month(s) old, for how long has he been in school?"
        answer = f"{years} years {months} months"

        wrong_answers = []
        generated_answers = {answer}

        while len(wrong_answers) < 3:
            diff_years = random.randint(-1, 1)
            diff_months = random.randint(-1, 2)

            wrong_years = years + diff_years
            wrong_months = months + diff_months

            # Adjusting years and months to ensure they remain in valid ranges
            if wrong_months > 11:
                wrong_months -= 12
                wrong_years += 1
            elif wrong_months < 0:
                wrong_months += 12
                wrong_years -= 1

            # Avoid generating a wrong answer that is outside the plausible range for school duration
            if wrong_years < 0 or wrong_years > 11:
                continue

            wrong_answer = f"{wrong_years} years {wrong_months} months"
            if wrong_answer not in generated_answers:
                wrong_answers.append(wrong_answer)
                generated_answers.add(wrong_answer)

    elif question_type == "medicine_times":
        medicine_interval = random.randint(40, 180) # Random interval between 40 and 180 minutes
        total_times = 24 * 60 // medicine_interval
        #pronoun = "he" if entity in ["a boy", "a man"] else "she"
        question = f"If a person is advised to take medicine every {medicine_interval} minutes, how many times will she take the medicine in a day?"
        answer = str(total_times)
        wrong_answers = set()
        while len(wrong_answers) < 3:
            wrong_answer = total_times + random.randint(-2, 2)
            if  wrong_answer >= 0 and wrong_answer != total_times:
                wrong_answers.add(str(wrong_answer))
        wrong_answers = list(wrong_answers)

    elif question_type == "homework_times":
        start_time = random.randint(1, 11)  # Adjusted to avoid 0
        is_midnight_start = random.choice([True, False])  # 50% chance to start at midnight (12 AM)
        end_time = random.randint(start_time + 1, 12)

        # Convert start_time to 12 AM if is_midnight_start is True
        start_time_str = "12 AM" if is_midnight_start else f"{start_time} AM"

        question = f"If a person starts doing homework at {start_time_str} and finishes at {end_time} PM, how many hours did he spend on homework?"

        correct_hours = end_time + 12 - (0 if is_midnight_start else start_time)  # Use 0 for 12 AM
        answer = f"{correct_hours} hours"

        wrong_answers = set()
        while len(wrong_answers) < 3:
            deviation = random.choice([-1, -2, 1, 2])  # choosing a random deviation to create incorrect answer
            wrong_answer_hours = correct_hours + deviation
            # Ensure the wrong answer is logical and not equal to the correct answer
            if 0 < wrong_answer_hours <= 24 and wrong_answer_hours != correct_hours:
                wrong_answers.add(f"{wrong_answer_hours} hours")
        wrong_answers = list(wrong_answers)


    elif question_type == "flight_times":
        departure_time = random.randint(0, 23)  # Random hour in the day
        duration = random.randint(1, 16)  # Random flight duration between 1 and 16 hours
        arrival_time = (departure_time + duration) % 24  # Calculate the arrival time in 24-hour format
            # Convert arrival time from 24-hour to 12-hour format and determine the suffix (AM/PM)
        if arrival_time >= 12:
            time_suffix = "PM"
            if arrival_time > 12:
                arrival_time -= 12
        else:
            time_suffix = "AM"
            if arrival_time == 0:
                arrival_time = 12

        question = f"If a flight takes off at {departure_time % 12 if departure_time % 12 else 12} {'AM' if departure_time < 12 else 'PM'} and the duration of the flight is {duration} hours, at what time will it land?"
        answer = f"{arrival_time} {time_suffix}"

        # Generate unique wrong answers
        wrong_answers_set = set()
        while len(wrong_answers_set) < 3:
            wrong_arrival_time = (arrival_time + random.randint(-4, 4)) % 12  # Random variation of the arrival time
            wrong_time_suffix = time_suffix if random.choice([True, False]) else ("PM" if time_suffix == "AM" else "AM")
            if wrong_arrival_time == 0:
                wrong_arrival_time = 12
            wrong_answers_set.add(f"{wrong_arrival_time} {wrong_time_suffix}")
        wrong_answers = list(wrong_answers_set)


    elif question_type == "rest_walk":
        speed = random.randint(5, 15) # Random speed between 5 and 15 km/hr
        rest_duration = random.randint(1, 5) # Rest duration between 1 and 5 minutes
        distance = random.randint(4, 10) # Distance between 4 and 10 km    
        total_time = (distance / speed * 60) + (distance - 1) * rest_duration   
        #pronoun = "he" if entity in ["a boy", "a man"] else "she"
        question = f"If a person walks at a speed of {speed} km/hr and after every km, she takes a rest for {rest_duration} minutes, how many minutes will it take her to cover {distance} km?"
        answer = str(int(total_time)) 
        wrong_answers_set = set()
        while len(wrong_answers_set) < 3:
            wrong_total_time = int(total_time + random.randint(-10, 10))

            # Ensure the wrong answer is non-negative and not equal to the correct answer
            if wrong_total_time > 0 and wrong_total_time != int(total_time):
                wrong_answers_set.add(str(wrong_total_time))
        wrong_answers = list(wrong_answers_set)
        
    elif question_type == "transportation":
        speed = random.randint(50, 800)
        distance = random.randint(50, 2000)

        time_hours = distance / speed
        answer = round(time_hours * 60)
        question = (f"A transportation operates with a constant speed of {speed} km/h. "
                    f"How long will it take to travel a distance of {distance} kilometers in minutes?")
        wrong_minutes = set()
        while len(wrong_minutes) < 3:
            deviation = random.choice([-15, -30, 15, 30])
            wrong_answer_minutes = answer + deviation

            # Ensure the wrong answer is logical and not equal to the correct answer
            if 0 < wrong_answer_minutes and wrong_answer_minutes != answer:
                wrong_minutes.add(f"{wrong_answer_minutes}")
        wrong_answers = list(wrong_minutes)


    options = [answer] + wrong_answers
    random.shuffle(options)
    return question, options, options.index(answer)


#df_practical = pd.DataFrame(columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Answer'])
data = []

for i in range(50):
    question, options, answer_index = generate_time_arithmetic_questions()
    answers = ['A', 'B', 'C', 'D']
    correct_answer = answers[answer_index]
    
    data.append({
        'Question': question,
        'Option A': options[0],
        'Option B': options[1],
        'Option C': options[2],
        'Option D': options[3],
        'Answer': correct_answer
    })

df_practical = pd.DataFrame(data)
df_practical = df_practical.drop_duplicates(subset=['Question'])

In [ ]:
df_time_arithmetic = pd.concat([df_24h, df_12h, df_year, df_month, df_month_year, df_date_op, df_month_day, df_week_day, df_week_in_year, df_time_zone, df_time_op, df_practical], ignore_index=True)